# Optional: House Prices using Regression

This notebook will focus on the application of Neural Networks for house price prediction. Instead of images as pixel-wise inputs, we will consider generic values that describe various features for houses.

![teaser](images/teaser.jpg)

In our previous exercises we omitted a detailed data preperation overview, as it was not the main focus of those exercises since it was disctracting your attention from the basics of neural networks. In this notebook, we will optionally look a little more in-depth in data analysis. In the end, the class will not focus tremendously on these tasks but they will be essential routines that you will encounter if you choose to work in the area of deep/machine learning.

The task itself is a *regression* problem and we will give you a few hinters how you can extend your build deep learning library of the first notebook to solve it, but you can also explore it as a *classification* task.

<div class="alert alert-danger">
    <h3>Warning</h3>
    <p>We will not explore neural network implementations in this exercise. If you want to train a network in the end, we suggest to work on "1_FullyConnectedNets.ipynb" first before starting the neural network part of this submission as we will be using the fully connected neural network class of the mentioned notebook.</p>
</div>

In [ ]:
# As usual, a bit of setup

import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

# House Price Data
## Exploration

Make sure to run the *download_datasets.sh* script first before running the upcoming cell. Previously, we provided you with a data loading wrapper function to access the CIFAR10 data. This time around, our input is a csv file which we will load ourselves using [pandas](https://pandas.pydata.org) where we can easily access and alter entries in our data matrix. Let's have a small glimpse how the data looks like!

In [ ]:
# Load the data
data = pd.read_csv("datasets/house_prices_data.csv")
labels = pd.read_csv("datasets/house_prices_labels.csv")

In [ ]:
#You can easily get an overview of our features using .info(). Note that not all features are actually numbers!
data.info()

In [ ]:
# Using the describe function we can get an overview about numerical ranges
data.describe()

In [ ]:
# Our target variable is the SalesPrice
# The pandas dataframe string it is data_train
# We explore it here
labels['SalePrice'].describe()

We will apply operations to a small subset of our data first.

In [ ]:
# Create smaller test data frames
data_small = data[:5]
labels_small = labels[:5]

## Transforms

In comparison to the previous notebooks, we always provided the data and label transforms for your convenience. However, if you want to apply your model to real life scenarios, you will have to provide a transform function that prepares raw data for your network. We already wrote a short transform class for you which will provide initial data augmenations for you under **exercise_code/transforms.py**. However, if you want to improve your network's performance, you will have to edit this class and change it to your liking!

In the end, you will not only submit your network model file but als your transform class! Using this class we can alter the test data as well as labels so that your neural network can correctly classify it.

In [ ]:
# Import the transform class
from exercise_code.transforms import Transforms

## Missing Data & Non-Numerical Values

Real life data is usually not perfect. There might be unreasonable or even missing entries. Let us first check out missing entries in our dataset.

In [ ]:
# Explore missing data
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# Only show top 20 entries
missing_data[:20]

As we have seen before, there are some columns that contains string information which can not parsed that easily. In order to use this information, one has to transform them into a (for the network) read-able format. 

For the initial setup, we will do the following two steps:
- we will omit non-numerical columns
- we will set all numerical missing values to 0

Those decisions are obviously not optimal. You are free to explore how one should use non-numerical entries to your advantage.

<div class="alert alert-info">
    <h3>Inline Question</h3>
    <p>What solution would you propose to handle missing numerical values? Do you think we can just transform non-numerical values to integers or should one do something more elaborate? In addition to that there might be new non-numerical entries that are not present in the training set. What would you propose how we should handle those?</p>
    <p>**Your answer:** </p>
</div>

In [ ]:
# For this notebook, we are performing a regression here for numerical attributes only
data_small = Transforms.get_only_numeric_attributes(data_small, verbose=True)
labels_small = Transforms.get_only_numeric_attributes(labels_small, verbose=True)

As you can see, we lost quite a bit of our features and are only left with 36 features.

## Normalization

As we have seen in our batch normalization notebook, it is of utmost important to properly normalize the input values for our network. We use our provided function.

In [ ]:
# Based on the min and max values that were learned on the training set,
# we are going to apply the same on the validation set
data_small = Transforms.min_max_scalar(data_small, invoker='data')
labels_small = Transforms.min_max_scalar(labels_small, invoker='label')

## Finished Transform Class

Our final transform class should have a single call to transform data as well as labels. This function will be called by our test as well and you should change it if you want to apply other transforms.

In [ ]:
# Create  a new small test set first
data_small = data[:5]
labels_small = labels[:5]

In [ ]:
# Check the full transform function for data as well as labels
prepared_data_small = Transforms.apply_data_transforms(data_small)
prepared_labels = Transforms.apply_labels_transforms(labels_small)

In [ ]:
# Lets have a short look to compare our prepared labels
print("Original labels:", labels_small)
print("Normalized labels:", prepared_labels)

## Data split

As for every training task we have to split up our provided data to validate our trained models.

In [ ]:
# Perform the split on data to create train and validation set
X_train = data[:800]
X_val = data[800:]
y_train = labels[:800]
y_val = labels[800:]
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

Check out the **min_max_scalar** function of the **Transforms** class. This function scales all values between zero and one as one can do for images. The difference here is that -- while images have a fixed minimum and maximum in 0 and 255 -- we don't know the theoretical minima and maxima for your data and thus our function is evoked differently for the training and validation/test set. 

You have to take those things into account for all transforms you'd like to implement!

In [ ]:
X_train = Transforms.apply_data_transforms(X_train, mode='train')
y_train = Transforms.apply_labels_transforms(y_train, mode='train')

In [ ]:
# Shapes of the training data
X_train.shape, y_train.shape

In [ ]:
# Convert the validation data using our now initialized transform class
X_val = Transforms.apply_data_transforms(X_val,mode='val')
y_val = Transforms.apply_labels_transforms(y_val, mode='val')

In [ ]:
X_val.shape, y_val.shape

Phew, that was a lot of work, but we are now done with our data preperation for now and can move to the actual network part!

# Next _optional_ steps:

## Classification of Mean

We are now ready to train networks. As a first step, you can simplify the task to a simple classification task that we can solve using our network structure from the first notebook.

In order to to this, we have to use different labels. We will alter our labels such that we return 1 if the entry is bigger than the mean of our training set or 0 if it is smaller. This can be done easily with our transform function.

### Data Preparation

In [ ]:
# Transform the labels into binary values since this is a classification task
def convert_to_binary_label(input_vector):
    """
    :param input_vector: a vector of real numbers
    :return: vector of 0,1 depending upon if the value is greater than mean value of the vector
    """
    mean_value = np.mean(input_vector)
    label_vector = np.array(input_vector > mean_value)
    return label_vector.astype(int)

In [ ]:
y_train_binary = convert_to_binary_label(y_train)
y_val_binary = convert_to_binary_label(y_val)

In [ ]:
# Convert values into a dictionary of the required format
# This dictionary can be fed into the NN
input_data = Transforms.prepare_dictionary(X_train, X_val, y_train_binary, y_val_binary)

### Network

Now it is your turn! Initialize a neural network as in the first notebook using the **FullyConnectedNet** class that you wrote prior. This task should not be that hard but as a sanity check you should first try to overfit on a smaller data first.

In [ ]:
# Import the previously used solver and network classes
from exercise_code.solver import Solver
from exercise_code.networks.fc_net import *

# ...

## A first look at: Regression

Previously, we approached the problem similarly to our previous tasks, i.e., we considered it like a classification problem. We can approximate the actual value by using more fine grained buckets but we have a hard time predicting an actual value.

For this task, we will explore regression to directly predict the actual numerical value. We have to make some changes to our loss function to use the **l2 loss**. Please take a look at the `l2_loss` function in `layers.py` for the implementation.

There are also some updates for the solver. As we directly predict float values, there is no easy notion of "accuracy". Thus, we will only consider the loss value and look for the model with the smallest loss.  Please take a look at `update_accuracy` function in `solver.py`.

In [ ]:
# Convert values into a dictionary of the required format
# This dictionary can be fed into the NN
input_data_for_NN = Transforms.prepare_dictionary(X_train, X_val, y_train, y_val)

In [ ]:
# ...

## Save the model

When you are satisfied with your training, you can save the model but there will be no submission for this notebook. The required number of submissions for this class have been reduced to 6 from 7.

<div class="alert alert-danger">
    <h3>Warning</h3>
    <p>You might get an error like this:</p>
    <p>PicklingError: Can't pickle <class 'exercise_code.classifiers.softmax.SoftmaxClassifier'>: it's not the same object as exercise_code.classifiers.softmax.SoftmaxClassifier</p>
    <p>The reason is that we are using autoreload and working on this class during the notebook session. If you get this error simply restart the kernel and rerun the whole script (Kernel -> Restart & Run All) or only the important cells for generating your model.</p>
</div>

In [ ]:
from exercise_code.model_savers import save_fully_connected_net
save_fully_connected_net(best_model, modelname='house_prices')